The first attempt to solve boston house price problem.

The dataset describes 13 numerical properties of houses in Boston suburbs and isc oncerned with modeling the price of the houses in those suburbs.

In [5]:
from __future__ import absolute_import, division, print_function

import numpy as np 
import pandas as pd 

from keras.models import Sequential 
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

In [13]:
# loading the dataset 
dataframe = pd.read_csv('housing.csv', delim_whitespace=True,header = None)
dataset = dataframe.values
print(dataset.shape)

(506, 14)


In [14]:
# spliting into input (X) and output (y) variables 

X = dataset[:,0:13]
Y = dataset[:,13]

The Keras wrappers require a function as an argument. This function that we must define is responsible for creating the neural network model to be evaluated.We are going to use a function that going to define to evaluate the baseline model. 

No activation function is used for the output layer because it is a regression problem and we are interested in predicting numerical values directly without transform.

In [15]:
# define base model
def baseline_model():
    # create model
    model = Sequential()
    model.add(Dense(13, input_dim=13, kernel_initializer='normal', activation='relu'))
    model.add(Dense(1, kernel_initializer='normal'))
    # Compile model
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

The Keras wrapper object for use in scikit-learn as a regression estimator is called KerasRegressor. We create an instance and pass it both the name of the function to create the neural network model as well as some parameters to pass along to the fit() function of the model later, such as the number of epochs and batch size. Both of these are set to sensible defaults.

We also initialize the random number generator with a constant random seed, a process we will repeat for each model evaluated in this tutorial. This is an attempt to ensure we compare models consistently.

In [19]:
# fix random seed for reproducibility
seed = 7
np.random.seed(seed)
# evaluate model with standardized dataset
estimator = KerasRegressor(build_fn=baseline_model,\
                           epochs=100, batch_size=5, verbose=0)

In [20]:
kfold = KFold(n_splits=10, random_state=seed)
results = cross_val_score(estimator, X, Y, cv=kfold)
print("Results: %.2f (%.2f) MSE" % (results.mean(), results.std()))

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Results: -32.69 (23.47) MSE


### Normalizing
An important concern with the Boston house price dataset is that the input attributes all vary in their scales because they measure different quantities.

In [22]:
# evaluate model with standardized dataset
np.random.seed(seed)
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasRegressor(build_fn=baseline_model,\
                   epochs=50, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = KFold(n_splits=10, random_state=seed)
results = cross_val_score(pipeline, X, Y, cv=kfold)
print("Standardized: %.2f (%.2f) MSE" % (results.mean(), results.std()))

Standardized: -29.26 (27.77) MSE
